In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
import json
import re
import requests
import pandas as pd

In [2]:
#Se crea una función para sacar la información de meme,TA, y la segunda persona que realiza la pull,si es que lo hubiera.
#Esta info se extrae de la parte de la apI de issues,que es donde se encuentra contenida,por eso se usa esta página,en lugar de la de pulls.

def getcontent(y,apiKey=os.getenv("GITHUB_APIKEY")):
    headers = {
        "Authorization": f"Bearer {apiKey}"
    }
    coment = requests.get(f'https://api.github.com/repos/ironhack-datalabs/datamad0820/issues/{y}/comments',headers=headers)
    coments = coment.json()
    return coments
    
#Se crean funciones con regex,para limpiar los campos que se obtiene de la función anterior,ya que sólo nos interesa
#parte de ella.Se crean varias funciones,una para cada dato de interes.

#2 usuario
def usuario_2(comment):
    try:
        return re.findall('@\w*-?\w+',comment[0]['body'])
    except:
        return None
#ta
def ta(comment):
    try:
        return comment[0]['user']['login']
    except:
        return None
#meme
def meme(comment):
    try:
        try:
            z = re.findall(r'https:.*jpg|.*png|.*jpeg',comment[0]['body'])
            z = str(z).split('(')
            z = z[1].split("'")
            return z[0]
        except: 
            z = re.findall(r'https:.*jpg|.*png|.*jpeg',comment[0]['body'])
            z = str(z).split('(')
            return z[0]
    except:
        return None   

#Se crea otra función para conseguir el resto de la info de las pulls,en este caso si dentro de la página de pulls.
#En esta nueva función se integra el return de la anterior,para que nos devuelva los datos de ambas apIs,en un mismo json.

def getdata(y,apiKey=os.getenv("GITHUB_APIKEY")):
    headers = {
        "Authorization": f"Bearer {apiKey}"
    }
    res = requests.get(f"https://api.github.com/repos/ironhack-datalabs/datamad0820/pulls/{y}", headers=headers)
    status = res.status_code
    data = res.json()  
    coments = getcontent(y)

    
    try:
        return {
            "Id":data['number'],
            "Estado":data["state"], 
            "Title":data["title"],
            "Usuario":data["user"]["login"],
            "Usuario_2" : usuario_2(coments),
            "Apertura":data["created_at"],
            "Cerrado":data["closed_at"],
            "TA":ta(coments),
            "Meme":meme(coments)}                    
    except:
        return {"ERROR":None}

In [17]:
#Se crea un bucle for para recorrer todas las pulls.
data=[getdata(i) for i in range(1,500)]
data

[{'Id': 1,
  'Estado': 'closed',
  'Title': '[lab-resolving-conflicts] Miguel Gimenez',
  'Usuario': 'miguelgimenezgimenez',
  'Usuario_2': None,
  'Apertura': '2020-08-17T13:53:33Z',
  'Cerrado': '2020-08-17T14:07:51Z',
  'TA': None,
  'Meme': None},
 {'Id': 2,
  'Estado': 'closed',
  'Title': '[lab-resolving-git-conflicts] Miguel Gimenez',
  'Usuario': 'miguelgimenezgimenez',
  'Usuario_2': [],
  'Apertura': '2020-08-17T14:10:42Z',
  'Cerrado': '2020-08-18T09:10:51Z',
  'TA': 'agalvezcorell',
  'Meme': 'https://user-images.githubusercontent.com/57899051/90494122-7332f800-e143-11ea-8527-f257dcbb6101.jpeg'},
 {'Id': 3,
  'Estado': 'closed',
  'Title': '[lab-resolving-git-conflicts] Irene Lopez',
  'Usuario': 'IreneLopezLujan',
  'Usuario_2': [],
  'Apertura': '2020-08-17T14:13:34Z',
  'Cerrado': '2020-08-18T09:39:55Z',
  'TA': 'WHYTEWYLL',
  'Meme': 'https://user-images.githubusercontent.com/49686519/90497587-8778f400-e147-11ea-8a36-9705614dbe14.png'},
 {'Id': 4,
  'Estado': 'closed',


In [21]:
#Convertimos en una cadena json para guardarlo en la base de datos después
jsonfile=json.dumps(data)

In [26]:
jsonfile = pd.DataFrame(data)
jsonfile.head(50)

,Id,Estado,Title,Usuario,Usuario_2,Apertura,Cerrado,TA,Meme,ERROR
0,1.0,closed,[lab-resolving-conflicts] Miguel Gimenez,miguelgimenezgimenez,None,2020-08-17T13:53:33Z,2020-08-17T14:07:51Z,None,None,NaN
1,2.0,closed,[lab-resolving-git-conflicts] Miguel Gimenez,miguelgimenezgimenez,[],2020-08-17T14:10:42Z,2020-08-18T09:10:51Z,agalvezcorell,https://user-images.githubusercontent.com/5789...,NaN
2,3.0,closed,[lab-resolving-git-conflicts] Irene Lopez,IreneLopezLujan,[],2020-08-17T14:13:34Z,2020-08-18T09:39:55Z,WHYTEWYLL,https://user-images.githubusercontent.com/4968...,NaN
3,4.0,closed,[lab-resolving-git-conflicts] Gontzal Monasterio,gontzalm,[],2020-08-17T14:29:27Z,2020-08-18T09:36:38Z,WHYTEWYLL,https://user-images.githubusercontent.com/4968...,NaN
4,5.0,closed,[lab-resolving-git-conflicts] Carlos Sanz,CarlosSanzDGP,[],2020-08-17T14:32:43Z,2020-08-18T09:31:26Z,WHYTEWYLL,https://user-images.githubusercontent.com/4968...,NaN
5,6.0,closed,[lab-resolving-git-conflicts] Silvia Hernandez,silviaherf,[],2020-08-17T14:42:18Z,2020-08-18T09:23:30Z,agalvezcorell,['https://user-images.githubusercontent.com/57...,NaN
6,7.0,closed,[lab-resolving-git-conflicts] Ana Murillo,AnaMA96,[],2020-08-17T15:03:15Z,2020-08-18T08:52:54Z,WHYTEWYLL,https://user-images.githubusercontent.com/4968...,NaN
7,8.0,closed,[lab-resolving-git-conflicts] Kevin Careaga,KevsDe,[],2020-08-17T15:03:22Z,2020-08-18T09:53:04Z,WHYTEWYLL,https://user-images.githubusercontent.com/4968...,NaN
8,9.0,closed,[lab-resolving-git-conflicts] Fernando Delgado,FDELTA,[],2020-08-17T15:05:27Z,2020-08-18T09:27:28Z,agalvezcorell,https://user-images.githubusercontent.com/5789...,NaN
9,10.0,closed,[lab-resolving-git-conflicts] - Marta Zavala,marta-zavala,[],2020-08-17T15:09:19Z,2020-08-18T10:01:50Z,WHYTEWYLL,[],NaN


In [27]:
jsonfile.to_json('output/jsonfile.json',orient="records")